In [2]:
import numpy as np

Source is from here: http://mlwiki.org/index.php/Gram-Schmidt_Process#QR_Factorization Approach is similar to the one used in gram-schmidt orthogonization.

In [3]:
def qr_factorization(A):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        v = A[:, j]

        for i in range(j - 1):
            q = Q[:, i]
            R[i, j] = q @ v
            v = v - R[i, j] * q

        norm = np.linalg.norm(v)
        Q[:, j] = v / norm
        R[j, j] = norm
    return Q, R

Example usage

In [4]:
np.random.seed(1)
# A = np.random.rand(13, 10) * 1000
A = np.array([[60, 91, 26], [60, 3, 75], [45, 90, 31]], dtype='float')
Q, R = qr_factorization(A)
Q.shape, R.shape

((3, 3), (3, 3))

In [5]:
Q

array([[ 0.62469505,  0.71080736, -0.63928383],
       [ 0.62469505,  0.02343321,  0.75368929],
       [ 0.46852129,  0.70299629, -0.15254061]])

In [6]:
R

array([[ 96.04686356,   0.        ,  77.61835966],
       [  0.        , 128.02343535,   0.        ],
       [  0.        ,   0.        ,  35.17655816]])

In [7]:
np.allclose(A, Q@R)

True

In [8]:
np.allclose(np.abs((Q, R)), np.abs(np.linalg.qr(A)))

False

In [9]:
np.linalg.qr(A)

(array([[-0.62469505,  0.35495981, -0.69552831],
        [-0.62469505, -0.76160078,  0.17239591],
        [-0.46852129,  0.54218796,  0.69750987]]),
 array([[ -96.04686356, -100.88825018,  -77.61835966],
        [   0.        ,   78.81345682,  -31.08327672],
        [   0.        ,    0.        ,   16.46876292]]))

In [10]:
np.abs((A - Q @ R).sum()) < 1e-6

True

Second version using Householder Transformation

In [11]:
def qr_householder(A):
    m, n = A.shape
    Q = np.eye(m) # Orthogonal transform so far
    R = A.copy() # Transformed matrix so far

    for j in range(n):
        # Find H = I - beta*u*u' to put zeros below R[j,j]
        x = R[j:, j]
        normx = np.linalg.norm(x)
        rho = -np.sign(x[0])
        u1 = x[0] - rho * normx
        u = x / u1
        u[0] = 1
        beta = -rho * u1 / normx

        R[j:, :] = R[j:, :] - beta * np.outer(u, u) @ R[j:, :]
        Q[:, j:] = Q[:, j:] - beta * Q[:, j:] @ np.outer(u, u)
        
    return Q, R

In [12]:
Q_h, R_h = qr_householder(A)
Q_h.shape, R_h.shape

((3, 3), (3, 3))

In [13]:
np.abs((A - Q_h @ R_h).sum()) < 1e-6

True

In [14]:
np.sqrt((12*12)+(6*6)+(4*4))

14.0

In [15]:
((2*2)+(6*6)+(4*4))

56

In [16]:
(1)+(3*3)+(2*2)

14

In [17]:
Q_h

array([[-0.62469505,  0.35495981,  0.69552831],
       [-0.62469505, -0.76160078, -0.17239591],
       [-0.46852129,  0.54218796, -0.69750987]])

In [18]:
R_h

array([[ -96.04686356, -100.88825018,  -77.61835966],
       [   0.        ,   78.81345682,  -31.08327672],
       [   0.        ,    0.        ,  -16.46876292]])

In [19]:
import numpy as np
from typing import Union
#source https://stackoverflow.com/questions/53489237/how-can-you-implement-householder-based-qr-decomposition-in-python

def householder(x: np.ndarray) -> Union[np.ndarray, int]:
    alpha = x[0]
    s = np.power(np.linalg.norm(x[1:]), 2)
    v = x.copy()

    if s == 0:
        tau = 0
    else:
        t = np.sqrt(alpha**2 + s)
        v[0] = alpha - t if alpha <= 0 else -s / (alpha + t)

        tau = 2 * v[0]**2 / (s + v[0]**2)
        v /= v[0]

    return v, tau

def householder_vectorized(a):
    """Use this version of householder to reproduce the output of np.linalg.qr 
    exactly (specifically, to match the sign convention it uses)

    based on https://rosettacode.org/wiki/QR_decomposition#Python
    """
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    tau = 2 / (v.T @ v)

    return v,tau

def qr_decomposition(A: np.ndarray):
    m,n = A.shape
    R = A.copy()
    Q = np.identity(m)

    for j in range(0, n):
        # Apply Householder transformation.
        v, tau = householder(R[j:, j])
        print(v)
        H = np.identity(m)
        H[j:, j:] -= tau * v.reshape(-1, 1) @ v
        R = H @ R
        Q = H @ Q

    return Q[:n].T, R[:n]

In [20]:
A

array([[60., 91., 26.],
       [60.,  3., 75.],
       [45., 90., 31.]])

In [21]:
# qr_decomposition(A)

In [22]:
m = 5
n = 4
A = np.random.rand(m, n)
q, r = np.linalg.qr(A)
# Q, R = qr_decomposition(A)

In [83]:
import numpy as np
#https://rosettacode.org/wiki/QR_decomposition#Python
def qr(A):
    m, n = A.shape
    Q = np.eye(m)
    for i in range(n - (m == n)):
        H = np.eye(m)
        H[i:, i:] = make_householder(A[i:, i])
        Q = Q@H
        A = H@A
    return Q, A
 
def make_householder(a):
    u = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    u[0] = 1
    H = np.eye(a.shape[0])
    H -= (2 / np.dot(u, u)) * u[:, None] @ u[None, :]
    return H
 
# task 1: show qr decomp of wp example
a = np.array(((
    (12, -51,   4),
    ( 6, 167, -68),
    (-4,  24, -41),
)))
 
q, r = qr(a)

In [67]:
q.round(4)

array([[ 0.9949,  0.0687, -0.0745],
       [ 0.0695,  0.0732,  0.9949],
       [ 0.0738, -0.995 ,  0.068 ]])

In [68]:
r.round(4)

array([[ 12.06  , -37.3672,  -3.7679],
       [  5.2426, -15.1617,  36.0926],
       [  4.8032, 171.5804, -70.7399]])

In [69]:
np.linalg.qr(a)

(array([[-0.85714286,  0.39428571,  0.33142857],
        [-0.42857143, -0.90285714, -0.03428571],
        [ 0.28571429, -0.17142857,  0.94285714]]),
 array([[ -14.,  -21.,   14.],
        [   0., -175.,   70.],
        [   0.,    0.,  -35.]]))

In [84]:
A = np.array([[60, 91, 26], [60, 3, 75], [45, 90, 31]], dtype='float')

In [93]:
Q, R = qr(A)

In [94]:
Q.round(3)

array([[-0.625,  0.355, -0.696],
       [-0.625, -0.762,  0.172],
       [-0.469,  0.542,  0.698]])

In [98]:
R.round(3)

array([[ -96.047, -100.888,  -77.618],
       [  -0.   ,   78.813,  -31.083],
       [   0.   ,    0.   ,   16.469]])

In [99]:
Q_np, R_np=np.linalg.qr(A)

In [101]:
np.allclose(Q, Q_np)

True

In [102]:
np.allclose(R, R_np)

True

## Solve linear equations

In [103]:
A = np.array([
[2., 1., 1.],
[1., 3., 2.],
[1., 0., 0]])
B = np.array([4., 5., 6.])

In [104]:
A

array([[2., 1., 1.],
       [1., 3., 2.],
       [1., 0., 0.]])

In [105]:
B

array([4., 5., 6.])

In [107]:
Q, R = np.linalg.qr(A) # QR decomposition with qr function
y = np.dot(Q.T, B) # Let y=Q'.B using matrix multiplication

In [108]:
R

array([[-2.44948974, -2.04124145, -1.63299316],
       [ 0.        , -2.41522946, -1.51814423],
       [ 0.        ,  0.        , -0.16903085]])

In [109]:
y

array([-7.75671752, -1.31112456,  3.88770957])

In [110]:
def back_substitution(U, y):
    
    #Number of rows
    n = U.shape[0]
    
    #Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    #Here we perform the back-substitution.  
    #Initializing with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    #Looping over rows in reverse (from the bottom up), 
    #starting with the second to last row, because the 
    #last row solve was completed in the last step.
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

In [111]:
back_substitution(R, y)

array([  6.,  15., -23.])

In [114]:
np.linalg.solve(R, y)

array([  6.,  15., -23.])

## Finding eigevalues

In [145]:
A = np.array([
[2., 1., 1.],
[1., 3., 2.],
[1., 0., 0]])
A

array([[2., 1., 1.],
       [1., 3., 2.],
       [1., 0., 0.]])

In [158]:
def find_eig_qr(A):
    pQ = np.eye(A.shape[0])
    X=A.copy()
    for i in range(100):
            Q,R = np.linalg.qr(X)
            pQ = pQ @ Q;
            X = R @ Q;
    return np.diag(X), pQ

In [159]:
find_eig_qr(A)

(array([ 3.91222918,  1.28646207, -0.19869124]),
 array([[ 0.51233387, -0.72297297, -0.46349121],
        [ 0.84874276,  0.50856627,  0.14490026],
        [ 0.13095702, -0.46762212,  0.87417379]]))

In [156]:
np.linalg.eig(A)

(array([ 3.91222918,  1.28646207, -0.19869124]),
 array([[-0.51233387, -0.51118241, -0.17058945],
        [-0.84874276,  0.76210326, -0.48349198],
        [-0.13095702, -0.39735522,  0.85856551]]))